In [4]:
import os

In [8]:
%pwd

'k:\\Environments\\End-End_ML_pipeline'

In [7]:
os.chdir("../")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
from DS_Ops.constants import *
from DS_Ops.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_url = config.source_url,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )
        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from DS_Ops import logger
from DS_Ops.utils.common import get_size

In [14]:
class DataIngestion:

    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: {headers}")

        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted all the files in {unzip_path}")


    

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    logger.exception(e)
    raise e

[2025-05-19 17:23:41,787]: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-19 17:23:41,789]: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-19 17:23:41,791]: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-19 17:23:41,791]: INFO: common: created directory at: ./artifacts]
[2025-05-19 17:23:41,791]: INFO: common: created directory at: ./artifacts/data_ingestion]


[2025-05-19 17:23:42,586]: INFO: 1895107213: artifacts\data_ingestion\winequality-red.zip download! with following info: Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "b92befccee872c4886a7387385b914c9a2652ab5bd49787562ef6946c157a90c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9294:3BD008:38FB9E:8FD645:682B1BC6
Accept-Ranges: bytes
Date: Mon, 19 May 2025 11:53:42 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100020-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1747655622.329518,VS0,VE287
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: c6b567b417fcef34124c7398e6fd9039078ed5ba
Expires: Mon, 19 May 2025 11:58:42 GMT
Source-Age: 0

]
[2025-05-19 17:23:42,631]: INFO: